# API'ing Eurostat and building maps with eurostatapiclient

Eurostat's API is a bit complicated to use. You can make use of their query builder to understand the structure: <br>
Query Builder: https://ec.europa.eu/eurostat/web/json-and-unicode-web-services/getting-started/query-builder<br>
But fortunately there is a Python wrapper: eurostatapiclient

In [ ]:
%matplotlib inline  

In [ ]:
!pip install -r requirements.txt

In case you don't use requirements.txt uncomment the following modules:

In [ ]:
#import pandas as pd
#from pandas.io.json import json_normalize
#import numpy as np
#import json
#import geopandas as gpd
#import matplotlib
#import matplotlib.pyplot as plt

#And now: The Eurostat API wrapper - find it here: https://github.com/opus-42/eurostat-api-client
#from eurostatapiclient import EurostatAPIClient

In [ ]:
VERSION = 'v2.1'

# only json supported
FORMAT = 'json'

# Specify language : en, fr, de
LANGUAGE = 'en'

# initializing
client = EurostatAPIClient(VERSION, FORMAT, LANGUAGE)

In [ ]:
# get the dataset by using Eurostat's ids
# e.g. from here: https://ec.europa.eu/eurostat/web/regions/data/database

id = 'tgs00101'

# You can define some filters 

params = {
    #'geo': 'DE',
    'time': '2017',
    #'sex' : 'F'
}

time = params['time']

filtered_dataset = client.get_dataset(id, params=params)


filtered_dataframe = filtered_dataset.to_dataframe()

In [ ]:
# extract the title of the dataset
title = filtered_dataset.label

In [ ]:
# take a quick look at the dataframe
# Column "geo" should show you the NUTS2 regional code
filtered_dataframe.head()

In [ ]:
filtered_dataframe.info()

In [ ]:
# finding max value
filtered_dataframe.loc[filtered_dataframe['values'].idxmax()]

# GeoJSON

Ok, let's start with the mappoing

In [ ]:
# Get the NUTS2 GeoJSON here: https://github.com/eurostat/Nuts2json
map_df = gpd.read_file("https://raw.githubusercontent.com/eurostat/Nuts2json/gh-pages/2016/3035/20M/2.json")


In [ ]:
# Rename the column "id" to "geo"
map_df['geo'] = map_df['id'] 

In [ ]:
map_df.head()

In [ ]:
# Quick plot of the GeoJSON
map_df.plot()

In [ ]:
# Map the corresponding NUTS2 columns from Eurostat's dataset and the GeoJSON and merge the two dataframes

merged = map_df.merge(filtered_dataframe, left_on='geo', right_on='geo')

merged.head(10)

In [ ]:
# set a variable that will call whatever column we want to visualise on the map

variable = 'values'

# set the range for the Choropleth colours

vmax = filtered_dataframe.loc[filtered_dataframe['values'].idxmax()]['values']
vmin = filtered_dataframe.loc[filtered_dataframe['values'].idxmin()]['values']

# Check
print(vmax, vmin)

# Optional: You could set vmin = 0
# vmin = 0

In [ ]:
# headline / title

headline = "Growing old in southern Europe"

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 8))

merged.plot(column='values', cmap='Blues', k=5, linewidth=0.3, ax=ax, edgecolor='0.9', vmin=vmin, vmax=vmax)  

# remove the Axis
ax.axis('off')

# Add a headline 
fig.suptitle(headline, fontsize=22, fontweight='bold')

# Use the Axis title for subtitle - e.g. the extracted title from above
ax.set_title(title, fontdict={'fontsize': '14', 'fontweight' : '3'})

annotation = "Map by XY/Media outlet, data source: Eurostat, " + time


# Add the Source
ax.annotate(annotation,xy=(0.1, .08),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=12, color='#555555')

# Create colorbar as a legend
sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))

# empty array for the data range
sm._A = []

# add the colorbar to the figure
cbar = fig.colorbar(sm)



In [ ]:
fig.savefig("map_export.png", dpi=300)
fig.savefig("map_export_illustrator.svg", dpi=300)